# <u>Chapter 4: Matrices</u>

Solving $\textbf{A}\vec{x}=\vec{b}$:
* First solve $\textbf{L}\vec{x}=\vec{b}$, where $\textbf{L}\equiv\text{lower triangular matrix}$. For $n=3$, <br><br>
$$\begin{bmatrix}
    L_{00} & 0 & 0\\
    L_{10} & L_{11} & 0\\
    L_{20} & L_{21} & L_{22}
\end{bmatrix}\begin{pmatrix}
    x_0\\
    x_1\\
    x_2
\end{pmatrix}=\begin{pmatrix}
    b_0\\
    b_1\\
    b_2
\end{pmatrix}$$

$$\begin{array}{}
    L_{00}x_0=b_0 & x_0=\dfrac{b_0}{L_{00}}\\
    L_{10}x_0+L_{11}x_1=b_1 & x_1=\dfrac{b_1-L_{10}x_0}{L_{11}}\\
    L_{20}x_{0}+L_{21}x_1+L_{22}x_2=b_2 & x_2=\dfrac{b_2-L_{20}x_0-L_{21}x_1}{L_{22}}
\end{array}$$

and so in general,
    $$x_i=\frac{b_i-\sum_{j=0}^{i-1}L_{ij}x_j}{L_{ii}}\text{ for }i=0,1,\ldots,n-1$$ <br>
this is called <b>forward substitution</b>. <br><br>
    - In the case of $\textbf{U}\equiv$ upper triangular matrix, $\textbf{U}\vec{x}=\vec{b}$. <br><br>
    - For $n=3$,
    $$\begin{bmatrix}
    U_{00} & U_{01} & U_{02}\\
    0 & U_{11} & U_{12}\\
    0 & 0 & U_{22}\end{bmatrix}
    \begin{pmatrix}
    x_0\\
    x_1\\
    x_2\end{pmatrix}=
    \begin{pmatrix}
    b_0\\
    b_1\\
    b_2\end{pmatrix}$$ <br>
$$\begin{array}{}
U_{00}x_0+U_{01}x_1+U_{02}x_2=b_0 & x_0=\frac{b_0-U_{01}x_1-U_{02}x_2}{U_{00}}\\
U_{11}x_1+U_{12}x_2=b_1 & x_1=\frac{b_1-U_{12}x_2}{U_{11}}\\
U_{22}x_2=b_2 & x_2=\frac{b_2}{U_{22}}
\end{array}$$ <br>
and so in general,
$$x_i=\frac{b_i-\sum_{j=i+1}^{n-1}U_{ij}x_j}{U_{ii}}\text{ for }i=n-1,n-2,\ldots,1,0$$ <br>
this is called <b>backward substitution</b>.
> <b>(Excerpt) Code 4.1:</b> <code>triang.py</code>

In [10]:
import numpy as np

def forsub(L,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in range(n):
        xs[i] = (bs[i] - L[i,:i]@xs[:i])/L[i,i]
    return xs

def backsub(U,bs):
    n = bs.size
    xs = np.zeros(n)
    for i in reversed(range(n)):
        xs[i] = (bs[i] - U[i,i+1:]@xs[i+1:])/U[i,i]
    return xs

def testcreate(n,val):
    A = np.arange(val, val+n*n).reshape(n,n)
    A = np.sqrt(A)
    bs = (A[0,:])**2.1
    return A, bs

def testsolve(f,A,bs):
    xs = f(A,bs)
    exs = np.linalg.solve(A,bs)
    print("(our solution - linalg.solve)/linalg.solve = \n",(xs-exs)/xs)

print("Testing forward sub solution")
A, bs = testcreate(4,21)
L = np.tril(A)
print(L)
print("bs = ",bs)
testsolve(forsub,L,bs)

print("\nTesting backward sub solution")
U = np.triu(A)
print(U)
print("bs = ",bs)
testsolve(backsub,U,bs)

Testing forward sub solution
[[4.58257569 0.         0.         0.        ]
 [5.         5.09901951 0.         0.        ]
 [5.38516481 5.47722558 5.56776436 0.        ]
 [5.74456265 5.83095189 5.91607978 6.        ]]
bs =  [24.45289367 25.67697243 26.90383729 28.13337297]
(our solution - linalg.solve)/linalg.solve = 
 [ 1.66448392e-16  1.41057642e-16 -1.22976849e-15  3.78841379e-15]

Testing backward sub solution
[[4.58257569 4.69041576 4.79583152 4.89897949]
 [0.         5.09901951 5.19615242 5.29150262]
 [0.         0.         5.56776436 5.65685425]
 [0.         0.         0.         6.        ]]
bs =  [24.45289367 25.67697243 26.90383729 28.13337297]
(our solution - linalg.solve)/linalg.solve = 
 [-7.05904414e-15  9.26805702e-15 -4.68367865e-15  0.00000000e+00]


* Gaussian Elimination: given a coefficient matrix representing a system of linear equations, the Gaussian Elimination is a sequence of operations performed to convert a coefficient matrix into an upper triangular matrix, known to be the matrix's row echelon form. <br><br>
    - Solving $\textbf{A}\vec{x}=\vec{b}$ with the Gaussian Elimination: <br><br>
$\underline{\mathcal{Ex:}}$ Given the following system of equations, <br><br>
$$\begin{array}{}
2x_0+x_1+x_2=8\\
x_0+x_1-2x_2=-2\\
5x_0+10x_1+5x_2=10\end{array}
\Longrightarrow\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
1 & 1 & -2 & -2\\
5 & 10 & 5 & 10\end{array}\right)$$ <br>
Method: we will define a new row $i^\prime\equiv i-\text{coeff}\times j$, where $j$ represents another row in the matrix called the pivot row. <br><br>
$$\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
1 & 1 & -2 & -2\\
5 & 10 & 5 & 10\end{array}\right)
\xrightarrow{i_1^\prime=i_1-\frac{1}{2}j_0}
\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
0 & \frac{1}{2} & -2\frac{1}{2} & -6\\
5 & 10 & 5 & 10\end{array}\right)$$ <br>
$$\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
0 & \frac{1}{2} & -2\frac{1}{2} & -6\\
5 & 10 & 5 & 10\end{array}\right)
\xrightarrow{i_2^\prime=i_2-\frac{5}{2}j_0}
\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
0 & \frac{1}{2} & -2\frac{1}{2} & -6\\
0 & 7\frac{1}{2} & 2\frac{1}{2} & -10\end{array}\right)$$ <br>
$$\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
0 & \frac{1}{2} & -2\frac{1}{2} & -6\\
0 & 7\frac{1}{2} & 2\frac{1}{2} & -10\end{array}\right)
\xrightarrow{i_2^\prime=i_2-15j_1}
\left(\begin{array}{ccc|c}
2 & 1 & 1 & 8\\
0 & \frac{1}{2} & -2\frac{1}{2} & -6\\
0 & 0 & 40 & 80\end{array}\right)$$ <br>
And so, <br><br>
$$\begin{array}{}
x_2=\frac{80}{40} & x_1=\frac{-6-(-2\frac{1}{2})x_2}{\frac{1}{2}} & x_0=\frac{8-x_1-x_2}{2}\end{array}$$ <br>
Observe: $\exists$ $n-2$ pivot rows, where $n$ represents the shape of the matrix (e.g., $j_0$ & $j_1$), and $\forall$ $j$, every row below are iterated each via elimination. <br><br>
$\underline{\mathcal{Ex:}}$ For $n=3\Longrightarrow j=\{j_0,j_1\}$, <br><br>
$$\begin{array}{}
\underline{j_0\text{:}} & \underline{j_1\text{:}}\\
i_1^\prime=i_1-\text{coeff}\times j_0 & i_2^\prime=i_2-\text{coeff}\times j_1\\
i_2^\prime=i_2-\text{coeff}\times j_0 & \;
\end{array}$$ <br>
Furthermore, it may be observed that the coefficients of each elimination is the quotient of the first elements of the pivot row over the iterated row <u>after</u> leading zeroes, so more precisely:
$$\begin{array}{}
\underline{j_0\text{:}} & \underline{j_1\text{:}}\\
i_1^\prime=i_1-\frac{A_{10}}{A_{00}}j_0 & i_2^\prime=i_2-\frac{A_{21}}{A_{11}}j_1\\
i_2^\prime=i_2-\frac{A_{20}}{A_{00}}j_0 & \;
\end{array}$$
And so in general, <br><br>
$$\text{coeff}=\frac{A_{ij}}{A_{jj}}\text{ for }\begin{array}{}
j=0,1,2,\ldots,n-2\\
i=j+1,j+2,\ldots,n-1\end{array}$$

> <b>(Excerpt) Code 4.2:</b> <code>gauelim.py</code>

In [11]:
import numpy as np

def gauelim(inA,inbs):
    A = np.copy(inA)
    bs = np.copy(inbs)
    n = bs.size
    
    for j in range(n-1):
        for i in range(j+1,n):
            coeff = A[i,j]/A[j,j]
            A[i,j:] -= coeff*A[j,j:]
            bs[i] -= coeff*bs[j]
            
    xs = backsub(A,bs)
    return xs

A, bs = testcreate(4,21)
testsolve(gauelim,A,bs)

(our solution - linalg.solve)/linalg.solve = 
 [-2.90829973e-09 -2.86313732e-09 -2.82428494e-09 -2.79126267e-09]


* LU Decomposition: <br><br>
    - We can decompose (with some caveats) a $n\times n$ matrix $\textbf{A}$ (i.e., a square matrix) into $\textbf{A}=\textbf{LU}$, where, <br><br> 
        * $\textbf{L}\equiv$ lower triangular matrix <br><br>
        * $\textbf{U}\equiv$ upper triangular matrix <br><br>
    - We will make the assumption that $\textbf{L}$ is a unit lower triangular matrix (i.e., $1$'s along its principal diagonal)

$\underline{\mathcal{Ex}:}$ Suppose $\textbf{A}$ from solving $\textbf{A}\vec{x}=\vec{b}$ during Gaussian Elimination, <br><br>
$$\textbf{A}=\begin{bmatrix}
2 & 1 & 1\\
1 & 1 & -2\\
5 & 10 & 5\end{bmatrix}
\Longrightarrow
\textbf{U}=\begin{bmatrix}
2 & 1 & 1\\
0 & \frac{1}{2} & -2\frac{1}{2}\\
0 & 0 & 40\end{bmatrix}$$ <br>
Then $\textbf{L}$ is a unit lower triangular matrix that contains the coefficients involved to solve each new row.
$$\textbf{L}=\left(\begin{array}{ccc|c}
1 & 0 & 0 & j+1\neq0\\
\frac{A_{10}}{A_{00}} & 1 & 0 & i_1^\prime\\
\frac{A_{20}}{A_{00}} & \frac{A_{21}}{A_{11}} & 1 & i_2^\prime
\end{array}\right)$$ <br>
(note: an augmented matrix was shown to keep track of each row entries corresponding to each new row created via elimination) <br><br>
So therefore,
$$\textbf{A}=\textbf{LU}\Longrightarrow
\begin{bmatrix}
2 & 1 & 1\\
1 & 1 & -2\\
5 & 10 & 5\end{bmatrix} = 
\begin{bmatrix}
2 & 1 & 1\\
0 & \frac{1}{2} & -2\frac{1}{2}\\
0 & 0 & 40\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0\\
\frac{1}{2} & 1 & 0\\
\frac{5}{2} & 15 & 1\end{bmatrix}$$

* Solving $\textbf{A}\vec{x}=\vec{b}$ via LU decomposition: <br><br>
    - Decompose $\textbf{A}=\textbf{LU}$, where $\textbf{L}\equiv$ lower triangular matrix and $\textbf{U}\equiv$ upper triangular matrix.
    $$\textbf{A}\vec{x}=\vec{b}\Longrightarrow\textbf{LU}\vec{x}=\vec{b}$$ <br>
    - Let $\textbf{U}\vec{x}=\vec{y}$:
    $$\textbf{LU}\vec{x}=\vec{b}\Longrightarrow\textbf{L}\vec{y}=\vec{b}$$ <br>
    - We may then solve: <br><br>
        * $\vec{y}$ in $\textbf{L}\vec{y}=\vec{b}$ via forward substitution. <br><br>
        * $\vec{x}$ in $\textbf{U}\vec{x}=\vec{y}$ via backward substitution.

> <b>(Excerpt) Code 4.3</b> <code>ludec.py</code>

In [18]:
import numpy as np

def ludec(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.identity(n)
    
    for j in range(n-1):
        for i in range(j+1,n):
            coeff = U[i,j]/U[j,j]
            U[i,j:] -= coeff*U[j,j:]
            L[i,j] = coeff
    return L, U

def lusolve(A,bs):
    L, U = ludec(A)
    ys = forsub(L,bs)
    xs = backsub(U,ys)
    return xs

if __name__ == '__main__':
    A, bs = testcreate(4,21)
    testsolve(lusolve,A,bs)

(our solution - linalg.solve)/linalg.solve = 
 [-2.90821111e-09 -2.86304959e-09 -2.82419811e-09 -2.79117672e-09]


In [22]:
c = 2
k = 20
w = 3
temp = []
temp.append(k-1*w**2)
temp.append(-k)
for j in range(0, 50-2):
    temp.append(0.)
np.array(temp, []).shape

(50,)